<a href="https://colab.research.google.com/github/uzh-rpg/rpg_timelens/blob/main/TimeLens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TimeLens: Event-based Video Frame Interpolation

## Installation

First clone the repo and install an environment

In [ ]:
# Check your current GPU
# If you are lucky, you get 16GB VRAM. If you are not lucky, you get less. VRAM is important. The more VRAM, the higher the maximum resolution will go
!nvidia-smi

Fri Sep 22 05:03:25 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Connect Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
print('Google Drive connected.')

Mounted at /content/gdrive
Google Drive connected.


In [ ]:
!ls /content/gdrive



MyDrive


In [ ]:
%%bash
rm -rf /content/rpg_timelens
rm -rf /content/ontent
rm -rf /content/sample_data
rm -rf /content/example

# Install Dependencies and Repo

In [ ]:
%cd /content
# !git clone https://github.com/uzh-rpg/rpg_timelens
# !git clone https://github.com/Chohoonhee/rpg_timelens
!git clone https://github.com/AI-Expert-Project/rpg_timelens
%cd /content/rpg_timelens
# !wget http://rpg.ifi.uzh.ch/timelens/data2/checkpoint.bin
!wget --load-cookies ~/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies ~/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1O43qlJMvmKQLMKlXlp42V7_Ww1r7sWio' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1O43qlJMvmKQLMKlXlp42V7_Ww1r7sWio" -O checkpoint.bin && rm -rf ~/cookies.txt

!CUDA_VISIBLE_DEVICES=0

/content
Cloning into 'rpg_timelens'...
remote: Enumerating objects: 146, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 146 (delta 28), reused 16 (delta 16), pack-reused 103
Receiving objects: 100% (146/146), 73.59 MiB | 3.36 MiB/s, done.
Resolving deltas: 100% (57/57), done.
/content/rpg_timelens
--2023-09-22 04:45:33--  https://docs.google.com/uc?export=download&confirm=t&id=1O43qlJMvmKQLMKlXlp42V7_Ww1r7sWio
Resolving docs.google.com (docs.google.com)... 74.125.68.113, 74.125.68.138, 74.125.68.139, ...
Connecting to docs.google.com (docs.google.com)|74.125.68.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-00-a8-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/8aj123tufta5ujjiiriorbi6i63fbf9l/1695357900000/18100762325334682086/*/1O43qlJMvmKQLMKlXlp42V7_Ww1r7sWio?e=download&uuid=dc5f943a-f9a6-4c7c-a5c5-fcd788e9fb3a [following]
--2023-09-22 04

In [ ]:
################# Required Configurations ############################

#@markdown # Required Configuration
#@markdown Use the values in here to configure what you'd like TimeLens to do. If you do not have your own data, just leave the values below and it will download an example.

#@markdown ## Output file
#@markdown Output file path: path (relative to the root of your Google Drive) for the output file. It will also determine the filetype in the destination. `.mp4` is recommended for video input, `.gif` for gif inputs.
OUTPUT_FILE_PATH = "/content/rpg_timelens/out.mp4" #@param{type:"string"}

################# Optional configurations ############################

#@markdown # Optional Configuration
#@markdown Parameters below can be left with their defaults, but feel free to adapt them to your needs.

#@markdown ## Target FPS
#@markdown  how many frames per second should the result have. This will determine how many intermediate images are interpolated.
TARGET_FPS =  960#@param{type:"number"}

#@markdown ## Frame input directory
#@markdown A path, relative to your GDrive root, where you already have the list of frames in the format 00001.png, 00002.png, etc.
FRAME_INPUT_DIR = '/content/rpg_timelens/input_frames' #@param{type:"string"}
EVENTS_INPUT_DIR = '/content/rpg_timelens/input_events' #@param{type:"string"}

#@markdown ## Frame output directory
#@markdown A path, relative to your GDrive root, where you want the generated frame.
FRAME_OUTPUT_DIR = '/content/rpg_timelens/output_frames' #@param{type:"string"}


# Set Up Example
If you do not have data to process, just leave the default values in the form above and the cell below will download an example.

In [ ]:
# Example

%%bash
wget http://rpg.ifi.uzh.ch/timelens/data2/example_github.zip -O /content/example_github.zip
unzip /content/example_github.zip -d /content/
rm -rf /content/example_github.zip
mv /content/example/events /content/rpg_timelens/input_events
mv /content/example/images /content/rpg_timelens/input_frames
mkdir /content/rpg_timelens/output_frames


Archive:  /content/example_github.zip
   creating: /content/example/
   creating: /content/example/events/
  inflating: /content/example/events/000358.npz  
  inflating: /content/example/events/000371.npz  
  inflating: /content/example/events/000368.npz  
  inflating: /content/example/events/000376.npz  
  inflating: /content/example/events/000383.npz  
  inflating: /content/example/events/000351.npz  
  inflating: /content/example/events/000353.npz  
  inflating: /content/example/events/000382.npz  
  inflating: /content/example/events/000365.npz  
  inflating: /content/example/events/000391.npz  
  inflating: /content/example/events/000354.npz  
  inflating: /content/example/events/000352.npz  
  inflating: /content/example/events/000356.npz  
  inflating: /content/example/events/000398.npz  
  inflating: /content/example/events/000389.npz  
  inflating: /content/example/events/000367.npz  
  inflating: /content/example/events/000381.npz  
  inflating: /content/example/events/000360

--2023-09-22 04:46:39--  http://rpg.ifi.uzh.ch/timelens/data2/example_github.zip
Resolving rpg.ifi.uzh.ch (rpg.ifi.uzh.ch)... 130.60.183.133
Connecting to rpg.ifi.uzh.ch (rpg.ifi.uzh.ch)|130.60.183.133|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://rpg.ifi.uzh.ch/timelens/data2/example_github.zip [following]
--2023-09-22 04:46:40--  https://rpg.ifi.uzh.ch/timelens/data2/example_github.zip
Connecting to rpg.ifi.uzh.ch (rpg.ifi.uzh.ch)|130.60.183.133|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://download.ifi.uzh.ch/rpg/web/data/timelens/data2/example_github.zip [following]
--2023-09-22 04:46:41--  https://download.ifi.uzh.ch/rpg/web/data/timelens/data2/example_github.zip
Resolving download.ifi.uzh.ch (download.ifi.uzh.ch)... 130.60.61.200
Connecting to download.ifi.uzh.ch (download.ifi.uzh.ch)|130.60.61.200|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10986

In [ ]:
# find the fps of the input images
import numpy as np
from os.path import join

timestamps = np.genfromtxt(join(FRAME_INPUT_DIR, "timestamp.txt"))
fps = int(1000000.0 / np.diff(timestamps).mean())
factor = int(TARGET_FPS / fps)
inserts = factor-1
print("Upsampling from ", fps, " to ", TARGET_FPS, " with ", inserts , " inserts.")



Upsampling from  159  to  960  with  5  inserts.


# TimeLens Networks

<img src = "https://drive.google.com/uc?id=10DOjPpDBfODA4rjjWew5EVyzfoBW58Mx" height = 300 width = 900>

<img src = "https://drive.google.com/uc?id=1aMlvxCFf8qq7Zz4UNdNJM3wBZCec5bs_" height = 300 width = 1100>


In [ ]:
import os
import sys

import pkg_resources

import torch as th
import torch.nn.functional as F
from timelens.common import size_adapter
from torch import nn

class down(nn.Module):
    def __init__(self, inChannels, outChannels, filterSize):
        super(down, self).__init__()
        self.conv1 = nn.Conv2d(
            inChannels,
            outChannels,
            filterSize,
            stride=1,
            padding=int((filterSize - 1) / 2),
        )
        self.conv2 = nn.Conv2d(
            outChannels,
            outChannels,
            filterSize,
            stride=1,
            padding=int((filterSize - 1) / 2),
        )

    def forward(self, x):
        x = F.avg_pool2d(x, 2)
        x = F.leaky_relu(self.conv1(x), negative_slope=0.1)
        x = F.leaky_relu(self.conv2(x), negative_slope=0.1)
        return x

class up(nn.Module):
    def __init__(self, inChannels, outChannels):
        super(up, self).__init__()
        # self.conv1 = nn.Conv2d(in_channels=?, out_channels=?, kernel_size=? , stride=?, padding=?)
        self.conv1 = nn.Conv2d(in_channels=inChannels, out_channels=outChannels, kernel_size=3, stride=1, padding=1)

        # self.conv2 = nn.Conv2d(in_channels=?, out_channels=outChannels, kernel_size=? , stride=?, padding=?)
        self.conv2 = nn.Conv2d(in_channels=outChannels * 2, out_channels=outChannels, kernel_size=3 , stride=1, padding=1)

    def forward(self, x, skpCn):
        x = F.interpolate(x, scale_factor=2, mode="bilinear")

        x = F.leaky_relu(self.conv1(x), negative_slope=0.1)
        x = F.leaky_relu(self.conv2(th.cat([x, skpCn], dim=1)), negative_slope=0.1) # concatenation
        return x


class UNet(nn.Module):
    """Modified version of Unet from SuperSloMo.

    Difference :
    1) there is an option to skip ReLU after the last convolution.
    2) there is a size adapter module that makes sure that input of all sizes
       can be processed correctly. It is necessary because original
       UNet can process only inputs with spatial dimensions divisible by 32.
    """

    def __init__(self, inChannels, outChannels, ends_with_relu=True):
        super(UNet, self).__init__()
        self._ends_with_relu = ends_with_relu
        self._size_adapter = size_adapter.SizeAdapter(minimum_size=32)
        self.conv1 = nn.Conv2d(inChannels, 32, 7, stride=1, padding=3)
        self.conv2 = nn.Conv2d(32, 32, 7, stride=1, padding=3)
        self.down1 = down(32, 64, 5)
        self.down2 = down(64, 128, 3)
        self.down3 = down(128, 256, 3)
        self.down4 = down(256, 512, 3)
        self.down5 = down(512, 512, 3)
        self.up1 = up(512, 512)
        self.up2 = up(512, 256)
        self.up3 = up(256, 128)
        self.up4 = up(128, 64)
        self.up5 = up(64,32)
        self.conv3 = nn.Conv2d(32, outChannels, 3, stride=1, padding=1)

    def forward(self, x):
        # Size adapter spatially augments input to the size divisible by 32.
        x = self._size_adapter.pad(x)
        x = F.leaky_relu(self.conv1(x), negative_slope=0.1)
        s1 = F.leaky_relu(self.conv2(x), negative_slope=0.1)
        s2 = self.down1(s1)
        s3 = self.down2(s2)
        s4 = self.down3(s3)
        s5 = self.down4(s4)
        x = self.down5(s5)
        x = self.up1(x, s5)
        x = self.up2(x, s4)
        x = self.up3(x, s3)
        x = self.up4(x, s2)
        x = self.up5(x, s1)

        # Note that original code has relu et the end.
        if self._ends_with_relu == True:
            x = F.leaky_relu(self.conv3(x), negative_slope=0.1)
        else:
            x = self.conv3(x)
        # Size adapter crops the output to the original size.
        x = self._size_adapter.unpad(x)
        return x


<img src = "https://drive.google.com/uc?id=10DOjPpDBfODA4rjjWew5EVyzfoBW58Mx" height = 300 width = 900>

In [ ]:
import torch as th
from timelens.common import warp
from timelens.common import pytorch_tools
from torch import nn


# ----------------------------------Warping-based Interpolation-----------------------------------------------
def _pack_voxel_grid_for_flow_estimation(example):
    return th.cat(
        [example["before"]["reversed_voxel_grid"], example["after"]["voxel_grid"]]
    )

def _pack_images_for_warping(example):
    return th.cat(
        [example["before"]["rgb_image_tensor"], example["after"]["rgb_image_tensor"]]
    )

class Warp(nn.Module):
    def __init__(self):
        super(Warp, self).__init__()
        self.flow_network = UNet(5, 2, False)

    def from_legacy_checkpoint(self, checkpoint_filename):
        checkpoint = th.load(checkpoint_filename)
        self.load_state_dict(checkpoint["networks"])

    def run_warp(self, example):
        flow = self.flow_network(_pack_voxel_grid_for_flow_estimation(example))
        warped, warped_invalid = warp.backwarp_2d(
            source=_pack_images_for_warping(example),
            y_displacement=flow[:, 0, ...],
            x_displacement=flow[:, 1, ...],
        )
        (before_flow, after_flow) = th.chunk(flow, chunks=2)
        (before_warped, after_warped) = th.chunk(warped, chunks=2)
        (before_warped_invalid, after_warped_invalid) = th.chunk(
            warped_invalid.detach(), chunks=2
        )
        return (
            before_warped,
            after_warped,
            before_flow,
            after_flow,
            before_warped_invalid,
            after_warped_invalid,
        )

    def run_and_pack_to_example(self, example):
        _pack_output_to_example(example, self.run_warp(example))

    def forward(self, example):
        return self.run_warp(example)
# ----------------------------------Interpolation Synthesis-----------------------------------------------
def _pack(example):
    return th.cat([example['before']['voxel_grid'],
                   example['before']['rgb_image_tensor'],
                   example['after']['voxel_grid'],
                   example['after']['rgb_image_tensor']], dim=1)

class Fusion(nn.Module):
    def __init__(self):
        super(Fusion, self).__init__()
        self.fusion_network = UNet(2*5 + 2*3, 3, False)

    def run_fusion(self, example):
        return self.fusion_network(_pack(example))

    def from_legacy_checkpoint(self, checkpoint_filename):
        checkpoint = th.load(checkpoint_filename)
        self.load_state_dict(checkpoint["networks"])

    def run_and_pack_to_example(self, example):
        example['middle']['fusion'] = self.run_fusion(example)

    def forward(self, example):
        return self.run_fusion(example)


# ----------------------------------Warping Refinement-----------------------------------------------

def _pack_for_residual_flow_computation(example):
    tensors = [
        example["middle"]["{}_warped".format(packet)] for packet in ["after", "before"]
    ]
    tensors.append(example["middle"]["fusion"])
    return th.cat(tensors, dim=1)


def _pack_images_for_second_warping(example):
    return th.cat(
        [example["middle"]["after_warped"], example["middle"]["before_warped"]],
    )

def _pack_output_to_example(example, output):
    (
        example["middle"]["before_refined_warped"],
        example["middle"]["after_refined_warped"],
        example["middle"]["before_refined_warped_invalid"],
        example["middle"]["after_refined_warped_invalid"],
        example["before"]["residual_flow"],
        example["after"]["residual_flow"],
    ) = output


class RefineWarp(Warp, Fusion):
    def __init__(self):
        Warp.__init__(self)
        self.flow_refinement_network = UNet(3*3 + 2*2, 2, False)

    def run_refine_warp(self, example):
        Warp.run_and_pack_to_example(self, example)
        Fusion.run_and_pack_to_example(self, example)
        residual = self.flow_refinement_network(
            _pack_for_residual_flow_computation(example)
        )
        (after_residual, before_residual) = th.chunk(residual, 2, dim=1)
        residual = th.cat([after_residual, before_residual], dim=0)
        refined, refined_invalid = warp.backwarp_2d(
            source=_pack_images_for_second_warping(example),
            y_displacement=residual[:, 0, ...],
            x_displacement=residual[:, 1, ...],
        )

        (after_refined, before_refined) = th.chunk(refined, 2)
        (after_refined_invalid, before_refined_invalid) = th.chunk(
            refined_invalid.detach(), 2)
        return (
            before_refined,
            after_refined,
            before_refined_invalid,
            after_refined_invalid,
            before_residual,
            after_residual,
        )


    def run_fast(self, example):
        Warp.run_and_pack_to_example(self, example)
        Fusion.run_and_pack_to_example(self, example)
        residual = self.flow_refinement_network(
            _pack_for_residual_flow_computation(example)
        )
        (after_residual, before_residual) = th.chunk(residual, 2, dim=1)
        residual = th.cat([after_residual, before_residual], dim=0)
        refined, _ = warp.backwarp_2d(
            source=_pack_images_for_second_warping(example),
            y_displacement=residual[:, 0, ...],
            x_displacement=residual[:, 1, ...],
        )

        return th.chunk(refined, 2)

    def run_and_pack_to_example(self, example):
        _pack_output_to_example(example, self.run_refine_warp(example))

    def forward(self, example):
        return self.run_refine_warp(example)


# -------------------------------------Attention-based averaging -----------------------------------------
def _pack_input_for_attention_computation(example):
    fusion = example["middle"]["fusion"]
    number_of_examples, _, height, width = fusion.size()
    return th.cat(
        [
            example["after"]["flow"],
            example["middle"]["after_refined_warped"],
            example["before"]["flow"],
            example["middle"]["before_refined_warped"],
            example["middle"]["fusion"],
            th.Tensor(example["middle"]["weight"])
            .view(-1, 1, 1, 1)
            .expand(number_of_examples, 1, height, width)
            .type(fusion.type()),
        ],
        dim=1,
    )


def _compute_weighted_average(attention, before_refined, after_refined, fusion):
    return (
        attention[:, 0, ...].unsqueeze(1) * before_refined
        + attention[:, 1, ...].unsqueeze(1) * after_refined
        + attention[:, 2, ...].unsqueeze(1) * fusion
    )


class AttentionAverage(RefineWarp):
    def __init__(self):
        Warp.__init__(self)
        self.attention_network = UNet(2*2 + 3*2 + 1, 3, False)

    def run_fast(self, example):
        example['middle']['before_refined_warped'],
        example['middle']['after_refined_warped'] = RefineWarp.run_fast(self, example)

        attention_scores = self.attention_network(
            _pack_input_for_attention_computation(example)
        )
        attention = F.softmax(attention_scores, dim=1)
        average = _compute_weighted_average(
            attention,
            example['middle']['before_refined_warped'],
            example['middle']['after_refined_warped'],
            example['middle']['fusion']
        )
        return average, attention

    def run_attention_averaging(self, example):
        RefineWarp.run_and_pack_to_example(self, example)
        attention_scores = self.attention_network(
            _pack_input_for_attention_computation(example)
        )
        attention = F.softmax(attention_scores, dim=1)
        average = _compute_weighted_average(
            attention,
            example["middle"]["before_refined_warped"],
            example["middle"]["after_refined_warped"],
            example["middle"]["fusion"],
        )
        return average, attention

    def run_and_pack_to_example(self, example):
        (
            example["middle"]["attention_average"],
            example["middle"]["attention"],
        ) = self.run_attention_averaging(example)

    def forward(self, example):
        return self.run_attention_averaging(example)


# Run TimeLens

In [ ]:
%cd /content/rpg_timelens/
!mkdir $FRAME_OUTPUT_DIR
!python -m timelens.run_timelens checkpoint.bin $EVENTS_INPUT_DIR $FRAME_INPUT_DIR $FRAME_OUTPUT_DIR 0 $inserts

/content/rpg_timelens
mkdir: cannot create directory ‘/content/rpg_timelens/output_frames’: File exists
Processing .
100% 50/50 [00:00<00:00, 216.31it/s]
Processing /content/rpg_timelens/output_frames/.
Counter: 0000
Events left:  6153 Events right:  31367
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Events left:  12482 Events right:  25038
Events left:  18783 Events right:  18737
Events left:  25037 Events right:  12483
Events left:  31292 Events right:  6228
Counter: 0006
Events left:  6314 Events right:  30593
Events left:  12529 Events right:  24378
Events left:  18630 Events right:  18277
Events left:  24734 Events right:  12173
Events left:  30844 Events right:  6063
Counter: 0012
Events left:  6036 Events right:  295

In [ ]:
!ffmpeg -i "$FRAME_OUTPUT_DIR"/%06d.png timelens.mp4
from google.colab import files
files.download("timelens.mp4")